Projeto Sistema de Monitoramento de Avanços no Campo da Genômica


DÚVIDA

-> Tem diferença (qual melhor) entre usar o requests e o cliente library ? https://newsapi.org/docs/client-libraries/python

1. Consumo de dados com a News API:

Implementar um mecanismo para consumir dados de notícias de fontes confiáveis e especializadas em genômica e medicina personalizada, a partir da News API:
https://newsapi.org/


In [0]:
import requests
from pyspark.sql.functions import *

API_KEY = '5220c4535529484381b6e56b7f95b0aa'

In [0]:
#Fazer request dos dados

s_tema = "genômica"
s_from = "2024-10-01"
s_sortBy = "publishedAt" # "publishedAt" or "popularity" or "relevancy"

url = ('https://newsapi.org/v2/everything?'
       f'q={s_tema}&'
       f'from={s_from}&'
       f'sortBy={s_sortBy}&'
       f'apiKey={API_KEY}')

print(url)
#response = requests.get(url)

https://newsapi.org/v2/everything?q=genômica&from=2024-10-01&sortBy=publishedAt&apiKey=5220c4535529484381b6e56b7f95b0aa


In [0]:
artigos = response.json()
print(artigos)

{'status': 'ok', 'totalResults': 5, 'articles': [{'source': {'id': None, 'name': 'Uol.com.br'}, 'author': 'Agência Fapesp', 'title': 'Bactéria multirresistente que representa risco global de saúde é detectada no Nordeste', 'description': 'Cepa de Klebsiella pneumoniae havia sido identificada previamente nos Estados Unidos. Microrganismo costuma causar infecções em ambiente hospitalar e não é eliminado por nenhum antibiótico existente, sendo particularmente perigoso para pessoas com baixa imuni…', 'url': 'https://gizmodo.uol.com.br/bacteria-multirresistente-que-representa-risco-global-de-saude-e-detectada-no-nordeste/', 'urlToImage': 'https://gizmodo.uol.com.br/wp-content/blogs.dir/8/files/2024/10/bacteria-fapesp.jpg', 'publishedAt': '2024-10-09T21:36:00Z', 'content': 'Texto: André Julião |Agência FAPESP\r\nUma cepa da bactéria Klebsiella pneumoniae isolada de uma mulher de 86 anos com infecção urinária, admitida em um hospital em uma cidade da região Nordeste em 202… [+5151 chars]'}, {

In [0]:
print(artigos["articles"][0].keys())

dict_keys(['source', 'author', 'title', 'description', 'url', 'urlToImage', 'publishedAt', 'content'])


In [0]:
#gravar em um df spark
lista_dados = []

for artigo in artigos["articles"]:
  artigo["source"] = artigo["source"]["name"]
  lista_dados.append(artigo)

df = spark.createDataFrame(lista_dados)

In [0]:
display(df)

author,content,description,publishedAt,source,title,url,urlToImage
Agência Fapesp,"Texto: André Julião |Agência FAPESP Uma cepa da bactéria Klebsiella pneumoniae isolada de uma mulher de 86 anos com infecção urinária, admitida em um hospital em uma cidade da região Nordeste em 202… [+5151 chars]","Cepa de Klebsiella pneumoniae havia sido identificada previamente nos Estados Unidos. Microrganismo costuma causar infecções em ambiente hospitalar e não é eliminado por nenhum antibiótico existente, sendo particularmente perigoso para pessoas com baixa imuni…",2024-10-09T21:36:00Z,Uol.com.br,Bactéria multirresistente que representa risco global de saúde é detectada no Nordeste,https://gizmodo.uol.com.br/bacteria-multirresistente-que-representa-risco-global-de-saude-e-detectada-no-nordeste/,https://gizmodo.uol.com.br/wp-content/blogs.dir/8/files/2024/10/bacteria-fapesp.jpg
Fábia Oliveira,"Sandy foi flagrada juntinho com o novo namorado, o médico Pedro Andrade, pela primeira vez. Os dois foram vistos abraçados andando pelas ruas da Europa. Na foto, obtida com exclusividade pelo perfil … [+813 chars]","A cantora foi fotografada pelas ruas do Europa abraçadinha com o novo amor; casal está junto há alguns meses, como revelou a coluna",2024-10-01T15:33:17Z,Metropoles.com,Foto: Sandy é flagrada com namorado médico pela primeira vez,https://www.metropoles.com/colunas/fabia-oliveira/foto-sandy-e-flagrada-com-namorado-medico-pela-primeira-vez,https://uploads.metroimg.com/wp-content/uploads/2024/08/12134715/Sandy-e-Pedro-Andrade.jpg
Gabriel Lima,"Ao que tudo indica, a relação entre Sandy e o médico Pedro Carvalho engrenou. Isso porque a cantora foi vista com o rapaz tomando um sorvete em Campinas, sua terra natal, ao lado do filho Theo, de 10… [+1994 chars]",A cantora Sandy está namorando o médico Pedro Andrade. Os dois estavam em uma sorveteira acompanhados do filho da artista,2024-10-01T13:30:02Z,Metropoles.com,"Sandy e médico circulam como um casal por Campinas, diz site",https://www.metropoles.com/celebridades/sandy-e-medico-circulam-como-um-casal-por-campinas-diz-site,https://uploads.metroimg.com/wp-content/uploads/2024/08/02093815/capa-1721.jpg
Fábia Oliveira,"A curiosidade dos fãs sobre o namoro de Sandy e Pedro Andrade, revelado com exclusividade por esta colunista no fim de julho, está cada vez maior, principalmente depois que uma foto dos pombinhos pas… [+3884 chars]","De acordo com Lucas Pasin, do UOL, a cantora Sandy já visitou o amado no trabalho e Pedro Andrade ainda deu bronca nos amigos por vazarem um vídeo",2024-10-02T14:30:29Z,Metropoles.com,Novos detalhes do namoro de Sandy e Pedro Andrade vêm à tona,https://www.metropoles.com/colunas/fabia-oliveira/novos-detalhes-do-namoro-de-sandy-e-pedro-andrade-vem-a-tona,https://uploads.metroimg.com/wp-content/uploads/2024/10/01123003/Sandy-e-Pedro-Andrade-1.jpg
Redação Terra,"Ao que parece o relacionamento de Sandy Leah Lima, de 41 anos, com o nutrólogo Pedro Andrade engrenou. Os dois foram vistos juntos nos últimos dias em uma sorveteria em Campinas (SP), terra natal da … [+2030 chars]","Ao que parece o relacionamento de Sandy Leah Lima, de 41 anos, com o nutrólogo Pedro Andrade ...",2024-10-01T12:54:31Z,Terra.com.br,Ficou sério? Sandy faz aparição pública com suposto novo namorado,"https://www.terra.com.br/diversao/gente/ficou-serio-sandy-faz-aparicao-publica-com-suposto-novo-namorado,6f1d0e045ee6b756838fb21a5c6d81bc1ffjxl31.html",https://p2.trrsf.com/image/fget/cf/1200/630/middle/images.terra.com/2024/08/02/medico-e-sandy-soejwwo7g2lw.png


In [0]:
df.printSchema()


root
 |-- author: string (nullable = true)
 |-- content: string (nullable = true)
 |-- description: string (nullable = true)
 |-- publishedAt: string (nullable = true)
 |-- source: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- urlToImage: string (nullable = true)




2. Definir Critérios de Relevância:
Desenvolver critérios precisos de relevância para filtrar as notícias. Por exemplo, o time pode se concentrar em notícias que mencionem avanços em sequenciamento de DNA, terapias genéticas personalizadas ou descobertas relacionadas a doenças genéticas específicas.


In [0]:
#Realizar os filtros necessários

In [0]:
#Salvar em parquet
#dbutils.fs.rm("genoma.parquet", True)
df.write.parquet('/FileStore/genoma.parquet')
display(spark.read.parquet('/FileStore/genoma.parquet'))

author,content,description,publishedAt,source,title,url,urlToImage
Agência Fapesp,"Texto: André Julião |Agência FAPESP Uma cepa da bactéria Klebsiella pneumoniae isolada de uma mulher de 86 anos com infecção urinária, admitida em um hospital em uma cidade da região Nordeste em 202… [+5151 chars]","Cepa de Klebsiella pneumoniae havia sido identificada previamente nos Estados Unidos. Microrganismo costuma causar infecções em ambiente hospitalar e não é eliminado por nenhum antibiótico existente, sendo particularmente perigoso para pessoas com baixa imuni…",2024-10-09T21:36:00Z,Uol.com.br,Bactéria multirresistente que representa risco global de saúde é detectada no Nordeste,https://gizmodo.uol.com.br/bacteria-multirresistente-que-representa-risco-global-de-saude-e-detectada-no-nordeste/,https://gizmodo.uol.com.br/wp-content/blogs.dir/8/files/2024/10/bacteria-fapesp.jpg
Redação Terra,"Ao que parece o relacionamento de Sandy Leah Lima, de 41 anos, com o nutrólogo Pedro Andrade engrenou. Os dois foram vistos juntos nos últimos dias em uma sorveteria em Campinas (SP), terra natal da … [+2030 chars]","Ao que parece o relacionamento de Sandy Leah Lima, de 41 anos, com o nutrólogo Pedro Andrade ...",2024-10-01T12:54:31Z,Terra.com.br,Ficou sério? Sandy faz aparição pública com suposto novo namorado,"https://www.terra.com.br/diversao/gente/ficou-serio-sandy-faz-aparicao-publica-com-suposto-novo-namorado,6f1d0e045ee6b756838fb21a5c6d81bc1ffjxl31.html",https://p2.trrsf.com/image/fget/cf/1200/630/middle/images.terra.com/2024/08/02/medico-e-sandy-soejwwo7g2lw.png
Fábia Oliveira,"A curiosidade dos fãs sobre o namoro de Sandy e Pedro Andrade, revelado com exclusividade por esta colunista no fim de julho, está cada vez maior, principalmente depois que uma foto dos pombinhos pas… [+3884 chars]","De acordo com Lucas Pasin, do UOL, a cantora Sandy já visitou o amado no trabalho e Pedro Andrade ainda deu bronca nos amigos por vazarem um vídeo",2024-10-02T14:30:29Z,Metropoles.com,Novos detalhes do namoro de Sandy e Pedro Andrade vêm à tona,https://www.metropoles.com/colunas/fabia-oliveira/novos-detalhes-do-namoro-de-sandy-e-pedro-andrade-vem-a-tona,https://uploads.metroimg.com/wp-content/uploads/2024/10/01123003/Sandy-e-Pedro-Andrade-1.jpg
Fábia Oliveira,"Sandy foi flagrada juntinho com o novo namorado, o médico Pedro Andrade, pela primeira vez. Os dois foram vistos abraçados andando pelas ruas da Europa. Na foto, obtida com exclusividade pelo perfil … [+813 chars]","A cantora foi fotografada pelas ruas do Europa abraçadinha com o novo amor; casal está junto há alguns meses, como revelou a coluna",2024-10-01T15:33:17Z,Metropoles.com,Foto: Sandy é flagrada com namorado médico pela primeira vez,https://www.metropoles.com/colunas/fabia-oliveira/foto-sandy-e-flagrada-com-namorado-medico-pela-primeira-vez,https://uploads.metroimg.com/wp-content/uploads/2024/08/12134715/Sandy-e-Pedro-Andrade.jpg
Gabriel Lima,"Ao que tudo indica, a relação entre Sandy e o médico Pedro Carvalho engrenou. Isso porque a cantora foi vista com o rapaz tomando um sorvete em Campinas, sua terra natal, ao lado do filho Theo, de 10… [+1994 chars]",A cantora Sandy está namorando o médico Pedro Andrade. Os dois estavam em uma sorveteira acompanhados do filho da artista,2024-10-01T13:30:02Z,Metropoles.com,"Sandy e médico circulam como um casal por Campinas, diz site",https://www.metropoles.com/celebridades/sandy-e-medico-circulam-como-um-casal-por-campinas-diz-site,https://uploads.metroimg.com/wp-content/uploads/2024/08/02093815/capa-1721.jpg


3. Cargas em Batches:
Armazenar as notícias relevantes em um formato estruturado e facilmente acessível para consultas e análises posteriores. Essa carga deve acontecer 1 vez por hora. Se as notícias extraídas já tiverem sidos armazenadas na carga anterior, o processo deve ignorar e não armazenar as notícias novamente, os dados carregados não podem ficar duplicados.


4. Dados transformados para consulta do público final
A partir dos dados carregados, aplicar as seguintes transformações e armazenar o resultado final para a consulta do público final:
4.1 - Quantidade de notícias por ano, mês e dia de publicação;
4.2 - Quantidade de notícias por fonte e autor;
4.3 - Quantidade de aparições de 3 palavras chaves por ano, mês e dia de publicação (as 3 palavras chaves serão as mesmas usadas para fazer os filtros de relevância do item 2 (2. Definir Critérios de Relevância)).
Atualizar os dados transformados 1 vez por dia.